In [9]:
dfStream = spark.readStream\
    .format("cosmos.oltp.changeFeed")\
    .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
    .option("spark.cosmos.container", "orders")\
    .option("spark.cosmos.changeFeed.startFrom", "Beginning")\
    .option("spark.cosmos.changeFeed.mode", "Incremental")\
    .load()

In [10]:
from   pyspark.sql.functions import * 
from pyspark.sql import *
from delta.tables import *

In [11]:
dfStream=dfStream.withColumn("Order_Date",to_date("Order_Timestamp"))

In [12]:
dfStream=dfStream.withColumn("Order_Timestamp",to_timestamp("Order_Timestamp"))

In [13]:
dfStream1=dfStream.withWatermark('Order_Timestamp', '10 minutes').groupBy("Item_Id","Order_Date",window("Order_Timestamp","8 minutes","4 minutes")).agg(sum("qty")).alias("sum_qty")

In [14]:
dfStream1=dfStream1.withColumnRenamed("sum(qty)","sum_qty")

In [15]:
def microbatch(batch_df,batch_id):
    delta_table=DeltaTable.forPath(spark, 'abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/tumbling_window')
    #print(delta_table)
    delta_table.alias('target').merge(batch_df.alias('updates'),'target.item_id = updates.item_id and target.window_Start_Time=updates.window_Start_Time and target.window_end_Time=updates.window_end_Time and target.order_date=updates.order_date').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [16]:
dfStream1.writeStream.outputMode("complete").format("delta").option("path","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/sliding_window").option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/sliding_window_checkpoint/").start()

In [17]:
#dfStream1.writeStream.outputMode("complete").foreachBatch(microbatch).option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/tumb_checkpoint/").start()

In [37]:
%%sql

select window.start,window.end,a.* from delta.`abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/sliding_window` a where item_id=10 order by 5 desc